In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math, scipy, skimage, os
import matplotlib as mpl
import seaborn as sns
from datetime import datetime

mpl.rcParams['font.family'] = 'Calibri'
mpl.rcParams['font.size'] = 12
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['pdf.fonttype'] = 42

In [2]:
# Write a function to calculate the nematic order
def nematic_order_filter(image, qmin=3, qmax=12, theta_bins=25):
    ys, xs = image.shape
    image_fft = scipy.fft.fft2(image)
    image_fft = scipy.fft.fftshift(image_fft)
    normalized_squared_modulus = (np.absolute(image_fft))**2
    
    # polar coordinates
    qx, qy = np.meshgrid(np.arange(ys)-(ys-1)/2, np.arange(xs)-(xs-1)/2)
    q = np.sqrt(qx**2 + qy**2)
    theta = np.arctan2(qy, qx)
    
    theta_bin = np.linspace(-np.pi, np.pi, int(theta_bins))
    # theta_bin_centers = np.unique(theta)

    h_theta = []
    for i in range(theta_bins-1):
        theta_condition = np.logical_and(theta>=theta_bin[i], theta<=theta_bin[i+1] )
        q_condition = np.logical_and(q>=qmin, q<=qmax)
        conditions = np.logical_and(theta_condition, q_condition)
        h_theta.append(np.sum(normalized_squared_modulus[conditions]))
        
    theta_bin_centers = (theta_bin[1:] + theta_bin[:-1])/2
    h_theta = np.array(h_theta)
    h_theta = h_theta/np.sum(h_theta)
    q_xx = -np.sum(h_theta * (np.cos(theta_bin_centers)**2 - 1/2))
    q_xy = -np.sum(h_theta * np.cos(theta_bin_centers) * np.sin(theta_bin_centers))
    
    return [q_xx, q_xy]

In [3]:
# Write a function to concert the first frame to NO
def movie_to_no(movie, pre_cut_frame_num=1, win_size=64, show_the_plot = True):
    _, ys, xs = movie.shape
    midy, midx = int(ys/2), int(xs/2)
    win_size = int(win_size/2)
    image = np.average(movie[0:pre_cut_frame_num], axis=0)
    image_center = image[midy-win_size:midy+win_size, midx-win_size:midx+win_size]

    qxx, _ = nematic_order_filter(image_center, qmin=1/30*win_size*2, qmax=15/30*win_size*2)
    # qxx, qxy = nematic_order_of_full_image(image_center, windowsize=win_size, qmin=3/30*win_size, qmax=12/30*win_size)
    # qxx = np.mean(qxx)
    # print(qxx)
    # print(np.mean(qxx))

    if show_the_plot:
        fig, axs = plt.subplots(1,2,figsize=[4,2])
        axs[0].imshow(image, cmap='Greys')
        rect = mpl.patches.Rectangle((midx-win_size, midy-win_size), win_size*2, win_size*2, linewidth=1, edgecolor='r', facecolor='none')
        axs[0].add_patch(rect)
        axs[1].imshow(image_center, cmap='Greys')
        axs[1].text(x = 0, y = 10, s = 'qxx = ' + str(round(qxx, 2)), color='red')
        plt.show()

    return qxx


Batch processing.

In [4]:
data_collection = pd.read_excel('B:\\home\\Xin home drive\\data_meroblastic\\majority_data_info.xlsx', sheet_name = 'lcii',
                                converters={'rep':int, 'cut_no':int, 'pre_cut_frame_num':int, 'post_cut_frame':int, 'no_analysis_method':str})
data_collection = data_collection.loc[data_collection['processed']=='processed']
data_collection

,rep,experiment,cut_type,batch,cut_no,where,orientation,note,processed,pixel_size,...,rv_analysis_method,rv_analysis_date,rv_manual,rv_manual_angle1,rv_manual_angle2,Unnamed: 31,recoiling_analysis1,recoiling_analysis2,recoiling_analysis3,recoiling_analysis4
62,62,xin250328_lcii_wt,wt,2.0,1,middle,perp,NaN,processed,0.16157,...,automatic,250723_1351,NaN,NaN,NaN,NaN,8.0,4.255,18.0,13.464
63,63,xin250328_lcii_wt,wt,2.0,2,edge,perp,NaN,processed,0.16157,...,automatic,250723_1351,NaN,NaN,NaN,NaN,8.0,2.908,18.0,7.378
64,64,xin250328_lcii_wt,wt,2.0,3,middle,perp,NaN,processed,0.16157,...,automatic,250723_1351,NaN,NaN,NaN,NaN,9.0,4.686,19.0,18.742
65,65,xin250328_lcii_wt,wt,2.0,4,middle,perp,NaN,processed,0.16157,...,automatic,250723_1351,NaN,NaN,NaN,NaN,8.0,7.513,18.0,23.508
66,66,xin250328_lcii_wt,wt,2.0,5,edge,perp,phase2,processed,0.16157,...,automatic,250723_1351,NaN,NaN,NaN,NaN,8.0,2.585,18.0,6.694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,1013,xin250710_lcii_wt,wt,2.0,59,ac,perp,NaN,processed,0.16157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1014,1014,xin250710_lcii_wt,wt,2.0,60,ac,para,NaN,processed,0.16157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015,1015,xin250710_lcii_wt,wt,2.0,61,ac,perp,NaN,processed,0.16157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1016,1016,xin250710_lcii_wt,wt,2.0,62,NaN,NaN,cut the septum with little recoiling,processed,0.16157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
replicates_experiments = data_collection['experiment'].unique()
print(replicates_experiments)

['xin250328_lcii_wt' 'xin250401_lcii_wt' 'xin250402_lcii_wt'
 'xin250403_lcii_wt' 'xin250408_lcii_wt' 'xin250411_lcii_wt'
 'xin250417_lcii_carhoa' 'xin250418_lcii_carhoa' 'xin250424_lcii_carhoa'
 'xin250625_lcii_carhoa' 'xin250626_lcii_carhoa' 'xin250703_lcii_wt'
 'xin250704_lcii_carhoa' 'xin250701_lcii_carhoa' 'xin250627_lcii_carhoa'
 'xin250710_lcii_wt']


In [4]:
math.degrees(math.atan(2.5/80))

1.7899106082460694

In [1]:
win_size = 40
print(win_size * 0.1615704)

win_size = int(win_size/2)

6.462816


In [7]:
for replicates_experiment in replicates_experiments:
    print(replicates_experiment)
    data_subset = data_collection.loc[data_collection['experiment']==replicates_experiment]

    j=0
    how_many_replicates = len(data_subset)

    for i in data_subset.index:
    # for i in [62]:
        replicate = data_collection.loc[i]
        j+=1
        print('rep', replicate.rep, "("+str(j)+"/"+str(how_many_replicates)+")")

        movie_file = replicate.experiment + "\\cut" + str(replicate.cut_no) + ".tif"
        if os.path.exists(movie_file):
            movie = skimage.io.imread(movie_file)
            pre_cut_frame_num = replicate.pre_cut_frame_num

            fs, ys, xs = movie.shape
            midy, midx = int(ys/2), int(xs/2)
            image = np.average(movie[0:pre_cut_frame_num], axis=0)
            image = skimage.exposure.rescale_intensity(image, in_range=tuple(np.percentile(image, (0,100))))
            image_center = image[midy-win_size:midy+win_size, midx-win_size:midx+win_size]

            qxx, qxy = nematic_order_filter(image_center, qmin=1/30*win_size*2, qmax=15/30*win_size*2, theta_bins=25)
            # qxx, qxy = nematic_order_of_full_image(image_center, windowsize=win_size, qmin=3/30*win_size, qmax=12/30*win_size)
            # qxx = np.mean(qxx)
            # print(qxx)
            # print(np.mean(qxx))

            if True:
                fig, axs = plt.subplots(1,2,figsize=[10, 5])
                axs[0].imshow(image, cmap='Greys_r', vmin=0, vmax=1)
                rect = mpl.patches.Rectangle((midx-win_size, midy-win_size), win_size*2, win_size*2, linewidth=1, edgecolor='yellow', facecolor='none')
                axs[0].add_patch(rect)
                axs[1].imshow(image_center, cmap='Greys_r', vmin=0, vmax=1)
                axs[1].text(x = 2, y = 5, s = 'qxx = ' + str(round(qxx, 2)), color='yellow')
                [ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False) for ax in axs]
                [spine.set_color('yellow') for spine in axs[1].spines.values()]
                [spine.set_linewidth(2) for spine in axs[1].spines.values()]
                plt.tight_layout()
                plt.savefig(movie_file[:-4] + "_no_analysis.png")
                # plt.show()
                plt.close()

            data_subset.loc[i, 'no_qxx'] = qxx
            data_subset.loc[i, 'no_qxy'] = qxy
            data_subset.loc[i, 'no_analysis_method'] = str(datetime.now().strftime(r"%y%m%d_%H%M") + "_40_01_15_25")
    
    data_collection_no = data_subset[['rep', 'no_qxx', 'no_qxy', 'no_analysis_method']]
    data_collection_no.to_excel("H:\\Xin home drive\\data_meroblastic\\Laser_cutter\\laser_cutter_ii\\" + replicates_experiment + "\\" + replicates_experiment + "_no_analysis_" + datetime.now().strftime(r"%y%m%d")+ ".xlsx")
    display(data_collection_no)
    print(data_collection_no[['no_qxx', 'no_qxy', 'no_analysis_method']].to_string(index=False))
    display('""""""""""""""""""""""""""""""""""""""""""""""""""""""""""')


xin250328_lcii_wt
rep 62 (1/45)
rep 63 (2/45)
rep 64 (3/45)
rep 65 (4/45)
rep 66 (5/45)
rep 67 (6/45)
rep 68 (7/45)
rep 69 (8/45)
rep 70 (9/45)
rep 71 (10/45)
rep 72 (11/45)
rep 73 (12/45)
rep 74 (13/45)
rep 75 (14/45)
rep 76 (15/45)
rep 77 (16/45)
rep 78 (17/45)
rep 79 (18/45)
rep 80 (19/45)
rep 81 (20/45)
rep 82 (21/45)
rep 83 (22/45)
rep 84 (23/45)
rep 85 (24/45)
rep 86 (25/45)
rep 87 (26/45)
rep 88 (27/45)
rep 89 (28/45)
rep 90 (29/45)
rep 91 (30/45)
rep 92 (31/45)
rep 93 (32/45)
rep 94 (33/45)
rep 95 (34/45)
rep 96 (35/45)
rep 97 (36/45)
rep 98 (37/45)
rep 99 (38/45)
rep 100 (39/45)
rep 101 (40/45)
rep 102 (41/45)
rep 103 (42/45)
rep 104 (43/45)
rep 105 (44/45)
rep 106 (45/45)


,rep,no_qxx,no_qxy,no_analysis_method
62,62,0.327778,-0.068917,250724_1333_40_01_15_25
63,63,0.111153,-0.066027,250724_1333_40_01_15_25
64,64,0.333716,-0.114240,250724_1333_40_01_15_25
65,65,0.364225,-0.011219,250724_1333_40_01_15_25
66,66,0.191305,0.074470,250724_1333_40_01_15_25
67,67,0.389991,-0.023211,250724_1333_40_01_15_25
68,68,0.340966,-0.190242,250724_1333_40_01_15_25
69,69,0.391402,-0.022197,250724_1333_40_01_15_25
70,70,0.113876,0.012190,250724_1333_40_01_15_25
71,71,0.221136,-0.101645,250724_1333_40_01_15_25


   no_qxx    no_qxy      no_analysis_method
 0.327778 -0.068917 250724_1333_40_01_15_25
 0.111153 -0.066027 250724_1333_40_01_15_25
 0.333716 -0.114240 250724_1333_40_01_15_25
 0.364225 -0.011219 250724_1333_40_01_15_25
 0.191305  0.074470 250724_1333_40_01_15_25
 0.389991 -0.023211 250724_1333_40_01_15_25
 0.340966 -0.190242 250724_1333_40_01_15_25
 0.391402 -0.022197 250724_1333_40_01_15_25
 0.113876  0.012190 250724_1333_40_01_15_25
 0.221136 -0.101645 250724_1333_40_01_15_25
 0.150903 -0.038302 250724_1333_40_01_15_25
 0.188058 -0.084491 250724_1333_40_01_15_25
 0.188020  0.005328 250724_1333_40_01_15_25
 0.183759 -0.020560 250724_1333_40_01_15_25
 0.255405 -0.062037 250724_1333_40_01_15_25
 0.019821  0.081411 250724_1333_40_01_15_25
 0.202160 -0.071381 250724_1333_40_01_15_25
 0.307060 -0.109985 250724_1333_40_01_15_25
 0.387097  0.034114 250724_1333_40_01_15_25
 0.080710 -0.052746 250724_1333_40_01_15_25
 0.267263 -0.035780 250724_1333_40_01_15_25
 0.331490 -0.062292 250724_1333_

'""""""""""""""""""""""""""""""""""""""""""""""""""""""""""'

xin250401_lcii_wt
rep 107 (1/78)
rep 108 (2/78)
rep 109 (3/78)
rep 110 (4/78)
rep 111 (5/78)
rep 112 (6/78)
rep 113 (7/78)
rep 114 (8/78)
rep 115 (9/78)
rep 116 (10/78)
rep 117 (11/78)
rep 118 (12/78)
rep 119 (13/78)
rep 120 (14/78)
rep 121 (15/78)
rep 122 (16/78)
rep 123 (17/78)
rep 124 (18/78)
rep 125 (19/78)
rep 126 (20/78)
rep 127 (21/78)
rep 128 (22/78)
rep 129 (23/78)
rep 130 (24/78)
rep 131 (25/78)
rep 132 (26/78)
rep 133 (27/78)
rep 134 (28/78)
rep 135 (29/78)
rep 136 (30/78)
rep 137 (31/78)
rep 138 (32/78)
rep 139 (33/78)
rep 140 (34/78)
rep 141 (35/78)
rep 142 (36/78)
rep 143 (37/78)
rep 144 (38/78)
rep 145 (39/78)
rep 146 (40/78)
rep 147 (41/78)
rep 148 (42/78)
rep 149 (43/78)
rep 150 (44/78)
rep 151 (45/78)
rep 152 (46/78)
rep 153 (47/78)
rep 154 (48/78)
rep 155 (49/78)
rep 156 (50/78)
rep 157 (51/78)
rep 158 (52/78)
rep 159 (53/78)
rep 160 (54/78)
rep 161 (55/78)
rep 162 (56/78)
rep 163 (57/78)
rep 164 (58/78)
rep 165 (59/78)
rep 166 (60/78)
rep 167 (61/78)
rep 168 (62/78)

...

'""""""""""""""""""""""""""""""""""""""""""""""""""""""""""'

xin250710_lcii_wt
rep 955 (1/63)
rep 956 (2/63)
rep 957 (3/63)
rep 958 (4/63)
rep 959 (5/63)
rep 960 (6/63)
rep 961 (7/63)
rep 962 (8/63)
rep 963 (9/63)
rep 964 (10/63)
rep 965 (11/63)
rep 966 (12/63)
rep 967 (13/63)
rep 968 (14/63)
rep 969 (15/63)
rep 970 (16/63)
rep 971 (17/63)
rep 972 (18/63)
rep 973 (19/63)
rep 974 (20/63)
rep 975 (21/63)
rep 976 (22/63)
rep 977 (23/63)
rep 978 (24/63)
rep 979 (25/63)
rep 980 (26/63)
rep 981 (27/63)
rep 982 (28/63)
rep 983 (29/63)
rep 984 (30/63)
rep 985 (31/63)
rep 986 (32/63)
rep 987 (33/63)
rep 988 (34/63)
rep 989 (35/63)
rep 990 (36/63)
rep 991 (37/63)
rep 992 (38/63)
rep 993 (39/63)
rep 994 (40/63)
rep 995 (41/63)
rep 996 (42/63)
rep 997 (43/63)
rep 998 (44/63)
rep 999 (45/63)
rep 1000 (46/63)
rep 1001 (47/63)
rep 1002 (48/63)
rep 1003 (49/63)
rep 1004 (50/63)
rep 1005 (51/63)
rep 1006 (52/63)
rep 1007 (53/63)
rep 1008 (54/63)
rep 1009 (55/63)
rep 1010 (56/63)
rep 1011 (57/63)
rep 1012 (58/63)
rep 1013 (59/63)
rep 1014 (60/63)
rep 1015 (61/63)

,rep,no_qxx,no_qxy,no_analysis_method
955,955,-0.015694,0.019925,250724_1342_40_01_15_25
956,956,-0.090165,-0.058319,250724_1342_40_01_15_25
957,957,0.025387,-0.043227,250724_1342_40_01_15_25
958,958,-0.077013,-0.024747,250724_1342_40_01_15_25
959,959,-0.049642,0.033641,250724_1342_40_01_15_25
...,...,...,...,...
1013,1013,-0.001217,-0.040585,250724_1343_40_01_15_25
1014,1014,0.042222,0.026594,250724_1343_40_01_15_25
1015,1015,0.043893,-0.017151,250724_1343_40_01_15_25
1016,1016,NaN,NaN,NaN


   no_qxx    no_qxy      no_analysis_method
-0.015694  0.019925 250724_1342_40_01_15_25
-0.090165 -0.058319 250724_1342_40_01_15_25
 0.025387 -0.043227 250724_1342_40_01_15_25
-0.077013 -0.024747 250724_1342_40_01_15_25
-0.049642  0.033641 250724_1342_40_01_15_25
 0.190512 -0.068935 250724_1342_40_01_15_25
-0.043894  0.089127 250724_1342_40_01_15_25
 0.040402 -0.049512 250724_1342_40_01_15_25
-0.014427 -0.069139 250724_1342_40_01_15_25
-0.024600  0.038892 250724_1342_40_01_15_25
 0.287334 -0.103017 250724_1342_40_01_15_25
-0.030068  0.008412 250724_1342_40_01_15_25
 0.010941  0.010385 250724_1342_40_01_15_25
-0.003906  0.014032 250724_1342_40_01_15_25
 0.017666 -0.018711 250724_1342_40_01_15_25
 0.222854 -0.201200 250724_1342_40_01_15_25
-0.000801 -0.057892 250724_1342_40_01_15_25
 0.071840 -0.064566 250724_1342_40_01_15_25
 0.310547 -0.155128 250724_1342_40_01_15_25
 0.008663 -0.035325 250724_1342_40_01_15_25
 0.008216  0.085198 250724_1342_40_01_15_25
-0.001433 -0.077593 250724_1342_

'""""""""""""""""""""""""""""""""""""""""""""""""""""""""""'